In [1]:
# Дано два файла разных форматов. Необходимо проанализировать их структуру и выделить общие хранимые данные. 
# Необходимо создать таблицу для хранения данных в базе данных. 
# Произведите запись данных из файлов разных форматов в одну таблицу. 
# Реализуйте и выполните следующие запросы:
#  вывод первых (VAR+10) отсортированных по произвольному числовому полю строк из таблицы в файл формата json;
#  вывод (сумму, мин, макс, среднее) по произвольному числовому полю;
#  вывод частоты встречаемости для категориального поля;
#  вывод первых (VAR+15) отфильтрованных по произвольному предикату отсортированных по произвольному числовому полю строк 
#  из таблицы в файл формате json.

In [1]:
import pandas as pd
import sqlite3
import pickle
import json
import csv

In [2]:
file_path = "task_3_var_17_part_1.csv"
data_csv = pd.read_csv(file_path, sep=';')
data_csv[0:5]

,artist,song,duration_ms,year,tempo,genre,energy,key,loudness
0,PSY,Gangnam Style (강남스타일),219493,2012,132.067,pop,0.937,11,-2.871
1,Sofía Reyes,"1, 2, 3 (feat. Jason Derulo & De La Ghetto)",201526,2018,94.968,"pop, latin",0.895,1,-3.112
2,Oliver Heldens,Gecko (Overdrive) - Radio Edit,165440,2014,124.959,"pop, Dance/Electronic",0.885,0,-5.469
3,Peking Duk,High (feat. Nicole Millar),228000,2014,100.094,Dance/Electronic,0.803,0,-4.052
4,Black Eyed Peas,Boom Boom Pow,251440,2009,130.048,"hip hop, pop",0.857,9,-5.892


In [3]:
with open('task_3_var_17_part_2.json', 'r', encoding='utf-8') as file:
    data_json = json.load(file)
data_json[0:3]

[{'artist': 'The Corrs',
  'song': 'Breathless',
  'duration_ms': '207506',
  'year': '2000',
  'tempo': '126.988',
  'genre': 'pop, Folk/Acoustic',
  'explicit': 'False',
  'popularity': '68',
  'danceability': '0.607'},
 {'artist': 'P!nk',
  'song': 'Get the Party Started',
  'duration_ms': '192533',
  'year': '2001',
  'tempo': '128.93',
  'genre': 'pop',
  'explicit': 'False',
  'popularity': '68',
  'danceability': '0.802'},
 {'artist': 'Jason Aldean',
  'song': 'Dirt Road Anthem',
  'duration_ms': '229413',
  'year': '2010',
  'tempo': '127.037',
  'genre': 'country',
  'explicit': 'False',
  'popularity': '53',
  'danceability': '0.678'}]

In [12]:
# Создание таблицы в базе данных
def create_table():
    conn = sqlite3.connect('database.db')
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS music
                  (id INTEGER PRIMARY KEY AUTOINCREMENT,
                   artist TEXT,
                   song TEXT,
                   duration_ms INTEGER,
                   year INTEGER,
                   tempo FLOAT,
                   genre TEXT)''')
    conn.commit()
    conn.close()

In [13]:
# Чтение данных из файлов в таблицу
def insert_data():
    conn = sqlite3.connect('database.db')
    c = conn.cursor()
    # Чтение данных из файла CSV и запись их в таблицу
    with open('task_3_var_17_part_1.csv', 'r', encoding='utf-8') as csv_file:
        csv_reader = csv.DictReader(csv_file, delimiter=';')
        for item in csv_reader:
            c.execute("INSERT INTO music (artist, song, duration_ms, year, tempo, genre) VALUES (?, ?, ?, ?, ?, ?)",
                      (item['artist'], item['song'], item['duration_ms'], item['year'], item['tempo'], item['genre']))
    # Чтение данных из файла JSON и запись их в таблицу
    with open('task_3_var_17_part_2.json', 'r', encoding='utf-8') as json_file:
        json_data = json.load(json_file)
        for data in json_data:
            c.execute("INSERT INTO music (artist, song, duration_ms, year, tempo, genre) VALUES (?, ?, ?, ?, ?, ?)",
                      (data['artist'], data['song'], data['duration_ms'], data['year'], data['tempo'], data['genre']))        
    conn.commit()
    conn.close()

In [14]:
# Вывод первых 27 отсортированных по числовому полю ['year'] в файл json
def sort_by_year():
    conn = sqlite3.connect('database.db')
    c = conn.cursor()
    c.execute("SELECT * FROM music ORDER BY year LIMIT 27")
    result = c.fetchall()
    with open('sorted_properties_3.json', 'w', encoding='utf-8') as file:
        json.dump(result, file, ensure_ascii=False)

    conn.close()  

In [15]:
# Вывод суммы, минимума, максимума и среднего значения по полю ['tempo']
def floors_statistics():
    conn = sqlite3.connect('database.db')
    c = conn.cursor()

    c.execute("SELECT SUM(tempo), MIN(tempo), MAX(tempo), AVG(tempo) FROM music")
    result = c.fetchone()

    conn.close()

    return result

In [16]:
# Вывод частоты встречаемости для категориального поля ['artist']
def city_frequency():
    conn = sqlite3.connect('database.db')
    c = conn.cursor()

    c.execute("SELECT artist, COUNT(*) FROM music GROUP BY artist")
    result = c.fetchall()

    conn.close()

    return result

In [17]:
# Вывод первых 32 отфильтрованных по произвольному предикату и отсортированных по числовому полю ['duration_ms'] в файл json
def filter_and_sort_by_views():
    conn = sqlite3.connect('database.db')
    c = conn.cursor()

    c.execute("SELECT * FROM music WHERE duration_ms > 10000 ORDER BY duration_ms LIMIT 32")
    result = c.fetchall()

    with open('filtered_properties_3.json', 'w', encoding='utf-8') as file:
        json.dump(result, file, ensure_ascii=False)

    conn.close()  

In [18]:
create_table()
insert_data()
sort_by_year()
floors_statistics()
city_frequency()
filter_and_sort_by_views()  